<a href="https://colab.research.google.com/github/sijuswamy/Test/blob/main/Object_Detection_using_YoloV5_on_Wound_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#  **Custom Training of YOLOV5 model on Wound dataset**

---
https://github.com/ultralytics/yolov5/wiki/Train-Custom-Data

https://blog.roboflow.com/how-to-train-yolov5-on-a-custom-dataset/




# 1.   Install Dependencies
Clone GitHub repository developed by  Ultralytics
, install dependencies and check PyTorch and GPU.



In [1]:
!git clone https://github.com/ultralytics/yolov5  # clone
%cd yolov5
%pip install -qr requirements.txt comet_ml  # install

import torch
import utils
display = utils.notebook_init()  # checks

YOLOv5 🚀 v7.0-399-g8cc44963 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)


Setup complete ✅ (2 CPUs, 12.7 GB RAM, 41.2/112.6 GB disk)


# 2. Mounting Wound Dataset from Google Drive.

Dataset: The wound dataset was collected from the AZH Wound and
Vascular Center, Milwaukee, WI, USA. This dataset (AZH
Wound Database) contains a total of 1,000 wound images of  Diabetic
foot ulcer (DFU) type. All the images are annotated and available in YOLO, VOC,COCO formats in the annotation directory. The
dataset is publicly available at https://github.com/uwmbigdata/wound_localization.

In [3]:

from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [15]:
# copy files
!cp '/content/gdrive/MyDrive/Yolo_Custom_V5/Data_annoted' "Data_annoted/content/custom_data/Data_annoted"

mv: cannot stat '/content/gdrive/MyDrive/Yolo_Custom_V5/Data_annoted': No such file or directory


In [6]:
# use this code if the the Data_annoted is a .zip file. Otherwise use previous code for moving images and annotations to custum_data folder
#!unzip "/content/gdrive/MyDrive/Yolo_Custom_V5/Data_annoted.zip" -d "/content/custom_data"

unzip:  cannot find or open /content/gdrive/MyDrive/Yolo_Custom_V5/Data_annoted.zip, /content/gdrive/MyDrive/Yolo_Custom_V5/Data_annoted.zip.zip or /content/gdrive/MyDrive/Yolo_Custom_V5/Data_annoted.zip.ZIP.


# 3.Data splitting
Dataset is split into train and validation set in the ration 80:20. The images are available in the location Custom data/ Data_annoted. The annotation files in yolo form should be saved in text format with the same name as that of the target image file. This text file are also available in the data_annoted folder.
After the data splitting the images and corresponding text file should be properly organized in the following directory location:

*   Images:

1.   Train Images: ..../images/train
2.   Validation Images: ../images/val




*   Labels:


1.   Train labels: ../labels/train
2.   Val labels: .../labels/val

This directory organization should be properly followed in order for the yolo model to fetch the images and labels





In [10]:
# creating paths
import os

# Define the paths
paths = [
    '/content/custom_data/images/train/',
    '/content/custom_data/labels/train/',
    '/content/custom_data/images/val/',
    '/content/custom_data/labels/val/',
]

# Create the directories
for path in paths:
    os.makedirs(path, exist_ok=True)

print("Folders created successfully!")


Folders created successfully!


In [14]:
import os
from random import choice
import shutil

# Arrays to store file names
imgs = []
xmls = []

# Setup directory names
trainPath = '/content/custom_data/images/train/'
trainlabelPath = '/content/custom_data/labels/train/'
valPath = '/content/custom_data/images/val/'
vallabelPath = '/content/custom_data/labels/val/'
#crsPath = '/content/custom_data/Data_annoted/'  # Directory where images and annotations are stored
crsPath = '/content/custom_data/'  # Directory where images and annotations are stored

# Setup ratios (val ratio = rest of the files in the origin dir after splitting into train and test)
train_ratio = 0.8
val_ratio = 0.2

# Total count of images
totalImgCount = len(os.listdir(crsPath)) / 2

# Sorting files into corresponding arrays
for (dirname, dirs, files) in os.walk(crsPath):
    for filename in files:
        if filename.endswith('.txt'):
            xmls.append(filename)
        else:
            imgs.append(filename)

# Counting range for cycles
countForTrain = int(len(imgs) * train_ratio)
countForVal = int(len(imgs) * val_ratio)

print("Training images are:", countForTrain)
print("Validation images are:", countForVal)

# Create the validation directory
os.makedirs(valPath, exist_ok=True)

# Cycle for train directory
for x in range(countForTrain):
    fileJpg = choice(imgs)  # Get the name of a random image from the origin dir
    fileXml = fileJpg[:-4] + '.txt'  # Get the name of the corresponding annotation file

    # Copy both files into the train directories
    shutil.copy(os.path.join(crsPath, fileJpg), os.path.join(trainPath, fileJpg))
    shutil.copy(os.path.join(crsPath, fileXml), os.path.join(trainlabelPath, fileXml))

    # Remove files from arrays
    imgs.remove(fileJpg)
    xmls.remove(fileXml)

# Cycle for validation directory
for x in range(countForVal):
    fileJpg = choice(imgs)  # Get the name of a random image from the origin dir
    fileXml = fileJpg[:-4] + '.txt'  # Get the name of the corresponding annotation file

    # Copy both files into the validation directories
    shutil.copy(os.path.join(crsPath, fileJpg), os.path.join(valPath, fileJpg))
    shutil.copy(os.path.join(crsPath, fileXml), os.path.join(vallabelPath, fileXml))

    # Remove files from arrays
    imgs.remove(fileJpg)
    xmls.remove(fileXml)


Training images are: 800
Validation images are: 200


#4. Edit the coco128.yaml file
coco128 yaml file is present in the yolov5 main directory under yolov5/data/coco128.yaml. The content of the file is as below..

*  path: ../datasets/coco128  # dataset root dir
*  train: images/train2017  # train images (relative to 'path') 128 images
*   val: images/train2017  # val images (relative to 'path') 128 images
*   test:  # test images (optional)

*   Classes (80 COCO classes) names:
  0: person
  1: bicycle
  2: car
  ...
  77: teddy bear
  78: hair drier
  79: toothbrush

  The above file need to edited by updating the paths as explained in the text box below..

In the updated coco128.yaml file for the wound data



*   path: ../custom_data/Data_annotated/
*   train:  ../images/train/
*   val: ...images/val/
*   test: ../images/test/ (optional)
*   classes: 1
*   0:wound


# 5.Custom Training of YOLOV5 on the wound dataset


In [16]:
# Train YOLOv5s on COCO128 for 3 epochs
!python train.py --img 640 --batch 16 --epochs 3 --data coco128.yaml --weights yolov5s.pt --cache

2025-03-23 06:38:22.050847: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1742711902.295415   14987 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1742711902.363523   14987 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
train: weights=yolov5s.pt, cfg=, data=coco128.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=3, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, evolve_population=data/hyps, resume_evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=exp, exist

# 6. Model Performace Evaluation and Visualization

Training results are automatically logged with Tensorboard and CSV loggers to runs/train, with a new experiment directory created for each new training as runs/train/exp2, runs/train/exp3, etc.

This directory contains train and val statistics, mosaics, labels, predictions and augmentated mosaics, as well as metrics and charts including precision-recall (PR) curves and confusion matrices.



In [18]:
from utils.plots import plot_results
plot_results('/content/yolov5/runs/train/exp/results.csv')  # plot 'results.csv' as 'results.png'

# Downloading the training results in zip fil(Optional)


In [ ]:
 !zip -r results.zip /content/yolov5/runs/train/exp4

  adding: content/yolov5/runs/train/exp4/ (stored 0%)
  adding: content/yolov5/runs/train/exp4/opt.yaml (deflated 49%)
  adding: content/yolov5/runs/train/exp4/labels_correlogram.jpg (deflated 43%)
  adding: content/yolov5/runs/train/exp4/val_batch0_pred.jpg (deflated 18%)
  adding: content/yolov5/runs/train/exp4/events.out.tfevents.1697612490.26286a88f5f1.3525.0 (deflated 42%)
  adding: content/yolov5/runs/train/exp4/confusion_matrix.png (deflated 39%)
  adding: content/yolov5/runs/train/exp4/val_batch1_pred.jpg (deflated 8%)
  adding: content/yolov5/runs/train/exp4/F1_curve.png (deflated 18%)
  adding: content/yolov5/runs/train/exp4/train_batch0.jpg (deflated 6%)
  adding: content/yolov5/runs/train/exp4/train_batch2.jpg (deflated 4%)
  adding: content/yolov5/runs/train/exp4/val_batch1_labels.jpg (deflated 8%)
  adding: content/yolov5/runs/train/exp4/PR_curve.png (deflated 25%)
  adding: content/yolov5/runs/train/exp4/val_batch2_pred.jpg (deflated 8%)
  adding: content/yolov5/runs/tra

# Detection
The test images are passed to the model to visualize the predicte output after training


In [19]:
!python detect.py --weights /content/yolov5/runs/train/exp/weights/best.pt --img 640 --conf 0.25 --source /content/gdrive/MyDrive/Yolo_Custom_V5/medetec/abdominal

detect: weights=['/content/yolov5/runs/train/exp/weights/best.pt'], source=/content/gdrive/MyDrive/Yolo_Custom_V5/medetec/abdominal, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_format=0, save_csv=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-399-g8cc44963 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)

Fusing layers... 
Model summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
image 1/11 /content/gdrive/MyDrive/Yolo_Custom_V5/medetec/abdominal/dehisced-surgical-wound-10.jpg: 448x640 1 wound, 37.5ms
image 2/11 /content/gdrive/MyDrive/Yolo_Custom_V5/medetec/abdominal/dehisced-surgical-wound-11.jpg: 480x640 4 wounds, 35.5ms
image 3/11 /